In [ ]:
import pandas as pd
import datetime as dt
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

# Tải các biến môi trường
load_dotenv()
print("Các thư viện và biến môi trường đã sẵn sàng.")

Các thư viện và biến môi trường đã sẵn sàng.


In [ ]:
# Lấy thông tin kết nối database
db_user = os.getenv("MYSQL_USER")
db_password = os.getenv("MYSQL_PASSWORD")
db_host = os.getenv("MYSQL_HOST")
db_port = os.getenv("MYSQL_PORT")
db_name = os.getenv("MYSQL_DATABASE")

# Tạo engine kết nối
connection_string = f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
engine = create_engine(connection_string)

# Đọc dữ liệu từ SQL vào DataFrame
query = "SELECT * FROM transactions"
df = pd.read_sql(query, engine)

# Chuyển đổi cột invoicedate sang đúng kiểu datetime (quan trọng cho việc tính toán)
df['invoicedate'] = pd.to_datetime(df['invoicedate'])

print(f"Đã tải thành công {len(df)} dòng dữ liệu giao dịch.")
df.head()

Đã tải thành công 397884 dòng dữ liệu giao dịch.


,ï»¿invoiceno,stockcode,description,quantity,invoicedate,unitprice,customerid,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


In [ ]:
# Tính cột revenue
df['revenue'] = df['quantity'] * df['unitprice']

# Xác định "ngày hôm nay" của bộ dữ liệu để tính Recency
# Ta sẽ lấy ngày giao dịch cuối cùng + 1 ngày
snapshot_date = df['invoicedate'].max() + dt.timedelta(days=1)
print(f"Ngày chốt dữ liệu (Snapshot Date): {snapshot_date}")

# Nhóm theo từng khách hàng và tính toán các giá trị RFM
# Sửa lại code với tên cột đúng cho cả invoiceno
rfm_df = df.groupby('customerid').agg({
    'invoicedate': lambda date: (snapshot_date - date.max()).days,
    'ï»¿invoiceno': 'nunique',  # <--- Sửa lại 'invoiceno' thành 'InvoiceNo'
    'revenue': 'sum'
})

# Đổi tên các cột cho dễ hiểu
rfm_df.rename(columns={
    'invoicedate': 'Recency',
    'ï»¿invoiceno': 'Frequency',
    'revenue': 'MonetaryValue'
}, inplace=True)

# Kiểm tra kết quả
print("Bảng RFM của khách hàng:")
rfm_df.head()

Ngày chốt dữ liệu (Snapshot Date): 2011-12-10 12:50:00
Bảng RFM của khách hàng:


,Recency,Frequency,MonetaryValue
customerid,,,
12346,326,1,77183.60
12347,2,7,4310.00
12348,75,4,1797.24
12349,19,1,1757.55
12350,310,1,334.40


In [ ]:
# Tạo cột 'Churn' dựa trên Recency
# Nếu Recency > 90 ngày, gán là 1 (Churn), ngược lại là 0 (Not Churn)
churn_threshold = 90
rfm_df['Churn'] = rfm_df['Recency'].apply(lambda x: 1 if x > churn_threshold else 0)

# Xem phân phối của nhãn Churn
print("Phân phối khách hàng Churn vs. Non-Churn:")
print(rfm_df['Churn'].value_counts(normalize=True))

rfm_df.head()

Phân phối khách hàng Churn vs. Non-Churn:
Churn
0    0.665975
1    0.334025
Name: proportion, dtype: float64


,Recency,Frequency,MonetaryValue,Churn
customerid,,,,
12346,326,1,77183.60,1
12347,2,7,4310.00,0
12348,75,4,1797.24,0
12349,19,1,1757.55,0
12350,310,1,334.40,1


In [ ]:
import os

# Đường dẫn đúng: đi lùi một cấp ('../'), sau đó vào data/processed
processed_data_path = '../data/processed/customer_features.csv'

# Lấy đường dẫn thư mục cha
directory = os.path.dirname(processed_data_path)

# Tạo thư mục nếu nó chưa tồn tại (vẫn giữ lại bước này cho chắc chắn)
os.makedirs(directory, exist_ok=True)

# Bây giờ, lệnh lưu file sẽ hoạt động đúng
rfm_df.to_csv(processed_data_path, index=False) # Thêm index=False để không lưu cột chỉ số

print(f"Dữ liệu đã được lưu thành công tại: {processed_data_path}")

OSError: Cannot save file into a non-existent directory: 'data\processed'